In [1]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.data.spatial.tile.manager import TileManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
from spectraclass.learn.manager import ClassificationManager, cm
from spectraclass.data.base import ModeDataManager
from spectraclass.data.spatial.modes import AvirisDataManager
from typing import List, Union, Tuple, Optional, Dict, Callable

2022-10-10 16:00:41.423921: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/gpfsm/ccds01/home/appmgr/app/jupyterhub/ilab/kernel/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  RequestsDependencyWarning)


Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [2]:
dm: DataManager = DataManager.initialize( "img_mgr", 'aviris' )
dm.modal.cache_dir = "/explore/nobackup/projects/ilab/cache"
dm.modal.data_dir = "/css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data/"

dm.modal.ext =  "_img"
dm.proc_type = "cpu"
TileManager.block_size = 125
ModeDataManager.model_dims =  16
TileManager.block_index = [1,4]
AvirisDataManager.valid_aviris_bands = [ [5,193], [214,283], [319,10000] ]

Opening log file:  '/home/tpmaxwel/.spectraclass/logging/aviris/img_mgr.log'
Using config file: '/panfs/ccds02/home/tpmaxwel/JupyterLinks/spectraclass-dev/defaults/config.py'
Using config file: '/home/tpmaxwel/.spectraclass/config/aviris/img_mgr.py'


/gpfsm/ccds01/home/appmgr/app/jupyterhub/ilab/kernel/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: UserWarning: Config option `refresh_data` not recognized by `DataManager`.
  interactivity=interactivity, compiler=compiler, result=result)
/panfs/ccds02/home/tpmaxwel/JupyterLinks/spectraclass-dev/notebooks/spectraclass/model/base.py:20: UserWarning: Config option `reduce_nepoch` not recognized by `AvirisDataManager`.  Did you mean one of: `reduce_method, reduce_nepochs`?
  inst = cls(*args, **kwargs)
/panfs/ccds02/home/tpmaxwel/JupyterLinks/spectraclass-dev/notebooks/spectraclass/model/base.py:20: UserWarning: Config option `reduce_niter` not recognized by `AvirisDataManager`.  Did you mean one of: `reduce_method, reduce_nepochs, reduce_sparsity`?
  inst = cls(*args, **kwargs)
/panfs/ccds02/home/tpmaxwel/JupyterLinks/spectraclass-dev/notebooks/spectraclass/model/base.py:20: UserWarning: Config option `refresh_model` not recognized by `AvirisDataManager`.
  inst = cl

Here we load the project data and define a set of class names and associated colors to be used in the classification process.

In [3]:
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue"),
            ('Class-5', "yellow")]

dm.loadCurrentProject()
lm().setLabels( classes )

Loading metadata from file: /explore/nobackup/projects/ilab/cache/spectraclass/aviris/img_mgr/ang20170731t222832rfl/ang20170731t222832_rfl_v2p9/ang20170731t222832_corr_v2p9_img.mdata.txt
Completed Reading raster file /css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data//ang20170731t222832rfl/ang20170731t222832_rfl_v2p9/ang20170731t222832_corr_v2p9_img, dims = ('band', 'y', 'x'), shape = (425, 14689, 698)
 Read point data, shape = (17100, 363), dims = ('samples', 'band')
 Preparing point data with shape (17100, 363), range=(-4.839442, 9.923917, -1.4048617e-09), #nan=0


AssertionError: Project initialization failed- check log file for details

Add Custom CNN

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras import datasets, layers, models
from spectraclass.learn.models.network import Network, ModelType

class CNN(Network):
    TYPE = ModelType.SPATIAL

    def _build_model(self, **kwargs) -> Tuple[Model,Dict]:
        from spectraclass.learn.models.spatial import SpatialModelWrapper
        nfeatures = kwargs.pop('nfeatures', 32 )
        from spectraclass.model.labels import lm
        input_shape = SpatialModelWrapper.get_input_shape()
        nclasses = lm().nLabels
        ks = kwargs.pop('kernel_size',3)
        model = models.Sequential()
        model.add( Input( shape=input_shape ) )
        model.add( layers.Conv2D( nfeatures, (ks,ks), activation='relu', padding="same" ) )
        model.add( layers.Reshape( SpatialModelWrapper.flatten(input_shape,nfeatures) ) )
        model.add( layers.Dense( nfeatures, activation='relu' ) )
 #       model.add( layers.Dense( nfeatures, activation='relu') )
        model.add( layers.Dense( nclasses, activation='softmax' ) )
        return model, kwargs

cm().addNetwork( CNN('cnn-1', nfeatures=64, nepochs=32 ) )

Here we start up the Spectraclass GUI.

In [ ]:
controller: SpectraclassController = app()
controller.gui()